# **Machine Learning Model Tuning and Evaluation**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint, uniform
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load dataset 
from sklearn.datasets import load_wine
wine = load_wine()
X = wine.data
y = wine.target

In [3]:
# spliting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Scaling the data set using Standard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
# Define models to evaluate
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier()
}

In [21]:
# Define hyperparameter grids for GridSearchCV
param_grids = {
    'Random Forest': {
        'n_estimators': [50, 60, 10],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVM': {
        'C': [0.1, 1, 105, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'Logistic Regression': {
        'C': [0.001, 0.01, 0.1, 1000, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    },
    'KNN': {
        'n_neighbors': [3, 5, 6, 8, 11],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    'Decision Tree': {
        'max_depth': [None, 15, 25, 30],
        'min_samples_split': [2, 3, 10],
        'min_samples_leaf': [1, 2, 9],
        'criterion': ['gini', 'entropy']
    }
}

In [22]:
# Define parameter distributions for RandomizedSearchCV
param_dists = {
    'Random Forest': {
        'n_estimators': randint(50, 250),
        'max_depth': [None] + list(np.arange(5, 50, 5)),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10)
    },
    'SVM': {
        'C': uniform(0.1, 100),
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 10))
    },
    'Logistic Regression': {
        'C': uniform(0.01, 150),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    },
    'KNN': {
        'n_neighbors': randint(1, 18),
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    'Decision Tree': {
        'max_depth': [None] + list(np.arange(5, 50, 5)),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'criterion': ['gini', 'entropy']
    }
}

In [16]:
# Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    return accuracy, precision, recall, f1

## Train and evaluate models with GridSearchCV

In [23]:
print("="*50)
print("GRID SEARCH CV RESULTS")
print("="*50)

grid_results = []

for name, model in models.items():
    print(f"\nTraining {name} with GridSearchCV...")
    start_time = time.time()
    
    grid_search = GridSearchCV(model, param_grids[name], cv=3, n_jobs=-1, verbose=0)
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    print("\nTest set performance:")
    accuracy, precision, recall, f1 = evaluate_model(grid_search.best_estimator_, X_test, y_test)
    
    training_time = time.time() - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    grid_results.append({
        'Model': name,
        'Best Score': grid_search.best_score_,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Training Time (s)': training_time
    })


GRID SEARCH CV RESULTS

Training Logistic Regression with GridSearchCV...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Best parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
Best cross-validation score: 0.9860

Test set performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

Confusion Matrix:
[[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
Training time: 1.95 seconds

Training Decision Tree with GridSearchCV...
Best parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best cross-validation score: 0.9223

Test set performance:
Accuracy: 0.9444
Precision: 0.9463
Recall: 0.9444
F1-Score: 0.9440

Classification Report:
              prec

## Train and evaluate models with RandomizedSearchCV

In [24]:
print("\n" + "="*60)
print("RANDOMIZED SEARCH CV RESULTS")
print("="*60)

random_results = []

for name, model in models.items():
    print(f"\nTraining {name} with RandomizedSearchCV...")
    start_time = time.time()
    
    random_search = RandomizedSearchCV(model, param_dists[name], n_iter=20, cv=5, n_jobs=-1, verbose=1, random_state=42)
    random_search.fit(X_train, y_train)
    
    print(f"Best parameters: {random_search.best_params_}")
    print(f"Best cross-validation score: {random_search.best_score_:.4f}")
    
    print("\nTest set performance:")
    accuracy, precision, recall, f1 = evaluate_model(random_search.best_estimator_, X_test, y_test)
    
    training_time = time.time() - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    random_results.append({
        'Model': name,
        'Best Score': random_search.best_score_,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Training Time (s)': training_time
    })



RANDOMIZED SEARCH CV RESULTS

Training Logistic Regression with RandomizedSearchCV...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters: {'C': 145.49647782429915, 'penalty': 'l2', 'solver': 'lbfgs'}
Best cross-validation score: 0.9722

Test set performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

Confusion Matrix:
[[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
Training time: 0.56 seconds

Training Decision Tree with RandomizedSearchCV...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'criterion': 'gini', 'max_depth': 35, 'min_samples_leaf': 1, 'min_samples_split': 13}
Best cross-validation score: 0.9224

Test set performance:
Accuracy: 0.9444
Precision: 0.9

In [25]:
# Comparing results
grid_df = pd.DataFrame(grid_results)
random_df = pd.DataFrame(random_results)

print("\n" + "="*50)
print("GRID SEARCH CV SUMMARY")
print("="*50)
print(grid_df.sort_values(by='Best Score', ascending=False))

print("\n" + "="*50)
print("RANDOMIZED SEARCH CV SUMMARY")
print("="*50)
print(random_df.sort_values(by='Best Score', ascending=False))


GRID SEARCH CV SUMMARY
                 Model  Best Score  Accuracy  Precision    Recall        F1  \
0  Logistic Regression    0.985963  1.000000   1.000000  1.000000  1.000000   
3        Random Forest    0.985816  1.000000   1.000000  1.000000  1.000000   
2                  SVM    0.978871  1.000000   1.000000  1.000000  1.000000   
4                  KNN    0.944001  0.944444   0.949383  0.944444  0.943604   
1        Decision Tree    0.922281  0.944444   0.946296  0.944444  0.943997   

   Training Time (s)  
0           1.949864  
3           8.058800  
2           0.097185  
4           0.215378  
1           0.208211  

RANDOMIZED SEARCH CV SUMMARY
                 Model  Best Score  Accuracy  Precision    Recall        F1  \
3        Random Forest    0.978571  1.000000   1.000000  1.000000  1.000000   
0  Logistic Regression    0.972167  1.000000   1.000000  1.000000  1.000000   
2                  SVM    0.971921  1.000000   1.000000  1.000000  1.000000   
4                

In [26]:
# Select best model based on F1 score
best_grid_model = grid_df.loc[grid_df['F1'].idxmax()]
best_random_model = random_df.loc[random_df['F1'].idxmax()]

print("\n" + "="*50)
print("BEST MODEL COMPARISON")
print("="*50)
print(f"Best GridSearchCV model: {best_grid_model['Model']} with F1-score {best_grid_model['F1']:.4f}")
print(f"Best RandomizedSearchCV model: {best_random_model['Model']} with F1-score {best_random_model['F1']:.4f}")




BEST MODEL COMPARISON
Best GridSearchCV model: Logistic Regression with F1-score 1.0000
Best RandomizedSearchCV model: Logistic Regression with F1-score 1.0000


In [27]:
# Final best model selection
if best_grid_model['F1'] > best_random_model['F1']:
    best_model_name = best_grid_model['Model']
    best_search = 'GridSearchCV'
    best_score = best_grid_model['F1']
else:
    best_model_name = best_random_model['Model']
    best_search = 'RandomizedSearchCV'
    best_score = best_random_model['F1']

print(f"\nOverall best model: {best_model_name} found by {best_search} with F1-score {best_score:.4f}")


Overall best model: Logistic Regression found by RandomizedSearchCV with F1-score 1.0000


## **🧾Conclusion**
For this dataset, the most dependable and computationally effective model without sacrificing accuracy or generalization was:

 <b>Logistic Regression</b>

Additionally, the pipeline uses GridSearchCV and RandomizedSearchCV to show off the effectiveness of hyperparameter tuning.
